In [1]:
#### Goal: predict GDP growth for 2011 using data from the past (time series). 
#### The table "EstimatedGDPGrowth" is where we have to store the predictions.
# Packages:
import sqlite3
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
#### First que list all objects in the database:
conn = sqlite3.connect('/home/ferran/Desktop/master/R&I/ml/final_project_skeleton/gdp_growth_forecast/db.sqlite3')
pd.read_sql(""" SELECT * FROM sqlite_master;""",conn)
        # "select *"": all columns.
        # "from": table (in this case, it's a summary).
        # "where column = filter".
#### load tables:
# Indicators: summary of variables.
# CountryIndicators: indicators per year & country (Y,covariates).
summ_vars = pd.read_sql("""SELECT * FROM Indicators;""",conn) #IndicatorName,IndicatorCode
long = pd.read_sql("""SELECT * FROM CountryIndicators;""",conn)



OperationalError: unable to open database file

In [5]:
#### 1) We find Y: EstimatedGDPGrowth for the next year:
summ_vars2 = summ_vars[summ_vars['IndicatorName'].str.contains("GDP")]
summ_vars2[~summ_vars2.IndicatorName.str.contains("% of GDP")]
    # Response variable: GDP growth (annual %): NY.GDP.MKTP.KD.ZG
    # GPD covariate: GDP (constant 2005 US$), NY.GDP.MKTP.KD [look description of response if any doubt].



,IndicatorCode,Topic,IndicatorName,ShortDefinition,LongDefinition,UnitOfMeasure,Periodicity,BasePeriod,OtherNotes,AggregationMethod,LimitationsAndExceptions,NotesFromOriginalSource,GeneralComments,Source,StatisticalConceptAndMethodology,DevelopmentRelevance,RelatedSourceLinks,OtherWebLinks,RelatedIndicators,LicenseType
294,NY.GDP.MKTP.KD.ZG,Economic Policy & Debt: National accounts: Growth rates,GDP growth (annual %),,Annual percentage growth rate of GDP at market prices based on constant local currency. Aggregates are based on constant 2005 U.S. dollars. GDP is the sum of gross value added by all resident producers in the economy plus any product taxes and minus any subsidies not included in the value of the products. It is calculated without making deductions for depreciation of fabricated assets or for depletion and degradation of natural resources.,,Annual,,,Weighted average,"Each industry's contribution to growth in the economy's output is measured by growth in the industry's value added. In principle, value added in constant prices can be estimated by measuring the quantity of goods and services produced in a period, valuing them at an agreed set of base year prices, and subtracting the cost of intermediate inputs, also in constant prices. This double-deflation method requires detailed information on the structure of prices of inputs and outputs.\n\nIn many industries, however, value added is extrapolated from the base year using single volume indexes of outputs or, less commonly, inputs. Particularly in the services industries, including most of government, value added in constant prices is often imputed from labor inputs, such as real wages or number of employees. In the absence of well defined measures of output, measuring the growth of services remains difficult.\n\nMoreover, technical progress can lead to improvements in production processes and in the quality of goods and services that, if not properly accounted for, can distort measures of value added and thus of growth. When inputs are used to estimate output, as for nonmarket services, unmeasured technical progress leads to underestimates of the volume of output. Similarly, unmeasured improvements in quality lead to underestimates of the value of output and value added. The result can be underestimates of growth and productivity improvement and overestimates of inflation.\n\nInformal economic activities pose a particular measurement problem, especially in developing countries, where much economic activity is unrecorded. A complete picture of the economy requires estimating household outputs produced for home use, sales in informal markets, barter exchanges, and illicit or deliberately unreported activities. The consistency and completeness of such estimates depend on the skill and methods of the compiling statisticians.\n\nRebasing of national accounts can alter the measured growth rate of an economy and lead to breaks in series that affect the consistency of data over time. When countries rebase their national accounts, they update the weights assigned to various components to better reflect current patterns of production or uses of output. The new base year should represent normal operation of the economy - it should be a year without major shocks or distortions. Some developing countries have not rebased their national accounts for many years. Using an old base year can be misleading because implicit price and volume weights become progressively less relevant and useful.\n\nTo obtain comparable series of constant price data for computing aggregates, the World Bank rescales GDP and value added by industrial origin to a common reference year. Because rescaling changes the implicit weights used in forming regional and income group aggregates, aggregate growth rates are not comparable with those from earlier editions with different base years. Rescaling may result in a discrepancy between the rescaled GDP and the sum of the rescaled components. To avoid distortio

In [6]:
#### 2) panel time series (wide format with only one country (all variables))
country = long[long['CountryCode'].str.contains("EUU")][["IndicatorCode","Year","Value"]]
df = country.pivot(index='IndicatorCode', columns='Year')
long.CountryCode.unique()

array(['ARB', 'CSS', 'CEB', 'EAS', 'EAP', 'EMU', 'ECS', 'ECA', 'EUU',
       'FCS', 'HPC', 'HIC', 'NOC', 'OEC', 'LCN', 'LAC', 'LDC', 'LMY',
       'LIC', 'LMC', 'MEA', 'MNA', 'MIC', 'NAC', 'OED', 'OSS', 'PSS',
       'SST', 'SAS', 'SSF', 'SSA', 'UMC', 'WLD', 'AFG', 'ALB', 'DZA',
       'ASM', 'ADO', 'AGO', 'ATG', 'ARG', 'ARM', 'ABW', 'AUS', 'AUT',
       'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLZ', 'BEN',
       'BMU', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN', 'BGR', 'BFA',
       'BDI', 'CPV', 'KHM', 'CMR', 'CAN', 'CYM', 'CAF', 'TCD', 'CHI',
       'CHL', 'CHN', 'COL', 'COM', 'ZAR', 'COG', 'CRI', 'CIV', 'HRV',
       'CUB', 'CUW', 'CYP', 'CZE', 'DNK', 'DJI', 'DMA', 'DOM', 'ECU',
       'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'ETH', 'FRO', 'FJI', 'FIN',
       'FRA', 'PYF', 'GAB', 'GMB', 'GEO', 'DEU', 'GHA', 'GRC', 'GRL',
       'GRD', 'GUM', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND', 'HKG',
       'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'IMY', 'ISR',
       'ITA', 'JAM',

In [20]:
#### 3) NA & corr analysis: df: 629 rows x 51 columns.
ncol = len(df.columns)
complete = df.apply(lambda x: x.count(), axis=1) #complete cases per row.
    # complete.quantile(np.arange(0.0, 1.1, 0.1))
# We keep 80% complete cases: 51*0.8 =40 
df2 = df[complete>40] # 184 rows x 51 columns
    # CAREFUL: For some countries de response has a lot of NA, we should check this.
mat_corr =df2.T.corr() # transpose matrix, then correlation.
corrY_sort= mat_corr['NY.GDP.MKTP.KD.ZG'].iloc[np.flip(mat_corr['NY.GDP.MKTP.KD.ZG'].abs().argsort())]
    # length of corrY_sort: 241
high_corr=corrY_sort[abs(corrY_sort) > 0.7] # abreviation & correlation.
translation_high_corr = summ_vars[summ_vars['IndicatorCode'].str.contains('|'.join(high_corr.index.tolist()))]
translation_high_corr

,IndicatorCode,Topic,IndicatorName,ShortDefinition,LongDefinition,UnitOfMeasure,Periodicity,BasePeriod,OtherNotes,AggregationMethod,LimitationsAndExceptions,NotesFromOriginalSource,GeneralComments,Source,StatisticalConceptAndMethodology,DevelopmentRelevance,RelatedSourceLinks,OtherWebLinks,RelatedIndicators,LicenseType
292,NE.EXP.GNFS.KD.ZG,Economic Policy & Debt: National accounts: Growth rates,Exports of goods and services (annual % growth),,"Annual growth rate of exports of goods and services based on constant local currency. Aggregates are based on constant 2005 U.S. dollars. Exports of goods and services represent the value of all goods and other market services provided to the rest of the world. They include the value of merchandise, freight, insurance, transport, travel, royalties, license fees, and other services, such as communication, construction, financial, information, business, personal, and government services. They exclude compensation of employees and investment income (formerly called factor services) and transfer payments.",,Annual,,,Weighted average,,,,"World Bank national accounts data, and OECD National Accounts data files.",,,,,,Open
294,NY.GDP.MKTP.KD.ZG,Economic Policy & Debt: National accounts: Growth rates,GDP growth (annual %),,Annual percentage growth rate of GDP at market prices based on constant local currency. Aggregates are based on constant 2005 U.S. dollars. GDP is the sum of gross value added by all resident producers in the economy plus any product taxes and minus any subsidies not included in the value of the products. It is calculated without making deductions for depreciation of fabricated assets or for depletion and degradation of natural resources.,,Annual,,,Weighted average,"Each industry's contribution to growth in the economy's output is measured by growth in the industry's value added. In principle, value added in constant prices can be estimated by measuring the quantity of goods and services produced in a period, valuing them at an agreed set of base year prices, and subtracting the cost of intermediate inputs, also in constant prices. This double-deflation method requires detailed information on the structure of prices of inputs and outputs.\n\nIn many industries, however, value added is extrapolated from the base year using single volume indexes of outputs or, less commonly, inputs. Particularly in the services industries, including most of government, value added in constant prices is often imputed from labor inputs, such as real wages or number of employees. In the absence of well defined measures of output, measuring the growth of services remains difficult.\n\nMoreover, technical progress can lead to improvements in production processes and in the quality of goods and services that, if not properly accounted for, can distort measures of value added and thus of growth. When inputs are used to estimate output, as for nonmarket services, unmeasured technical progress leads to underestimates of the volume of output. Similarly, unmeasured improvements in quality lead to underestimates of the value of output and value added. The result can be underestimates of growth and productivity improvement and overestimates of inflation.\n\nInformal economic activities pose a particular measurement problem, especially in developing countries, where much economic activity is unrecorded. A complete picture of the economy requires estimating household outputs produced for home use, sales in informal markets, barter exchanges, and illicit or deliberately unreported activities. The consistency and completeness of such estimates depend on the skill and methods of the compiling statisticians.\n\nRebasing of national accounts can alter the measured growth rate of an economy and lead to breaks in series that affect the consistency of data over time. When countries rebase their national accounts, they update the weights assigned to various components to better reflect current patterns of production o

In [ ]:
#### Helpful comments:
# I want to predict year 2010 bc is what we have.
# we must include in the model Y_{t-1}.
# First we should do feature selection (lasso), then NA imputation, then prediction (quantile regression or gradient boosting.

This is just a Markdown